In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import plotly.io as pio
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import matplotlib.font_manager as fm
font_path = './AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()

In [2]:
iem = pd.read_csv("iem_new2.csv", encoding='cp949')
cus =pd.read_csv("cus_info_new.csv", encoding='cp949')
hist = pd.read_csv("stk_bnc_hist.csv", encoding='cp949')
train_eco = pd.read_csv("train_경제지표.csv", encoding='cp949')
test = pd.read_csv("test_경제지표.csv", encoding='cp949')

In [3]:
cus.columns=["계좌 ID","성별","연령대","투자성향","자산구간","주거래상품군","Life Style","서비스 등급","총 투자기간","주거래업종구분"]
hist.columns= ["계좌 ID","기준일자","종목코드","잔고수량","잔고금액","주당 액면가"]

In [4]:
hist['기준일자'] = hist['기준일자'].astype('str')
hist['기준일자'] = hist['기준일자'].apply(lambda x: str(x[0:4]) + '-' + str(x[4:6]) + '-'+ str(x[6:8]))

In [5]:
train_eco_scaled = pd.read_csv('train_eco_scaled.csv', encoding='cp949')

# hist_d 생성

## 연도별로 0.1씩

In [5]:
train_eco['year']= train_eco['매수일자'].apply(lambda x: x[0:4])

In [6]:
train_eco['과거 보유일'] = np.nan

train_eco2016 = train_eco[train_eco['year']=='2016'].copy()
train_eco2016['과거 보유일'] = train_eco2016['보유기간(일)']*0.5

train_eco2017 = train_eco[train_eco['year']=='2017'].copy()
train_eco2017['과거 보유일'] = train_eco2017['보유기간(일)']*0.6

train_eco2018 = train_eco[train_eco['year']=='2018'].copy()
train_eco2018['과거 보유일'] = train_eco2018['보유기간(일)']*0.7

train_eco2019 = train_eco[train_eco['year']=='2019'].copy()
train_eco2019['과거 보유일'] = train_eco2019['보유기간(일)']*0.8

train_eco2020 = train_eco[train_eco['year']=='2020'].copy()
train_eco2020['과거 보유일'] = train_eco2020['보유기간(일)']*0.9

In [7]:
train_year =pd.concat([train_eco2016,train_eco2017,train_eco2018,train_eco2019,train_eco2020], ignore_index=True)

In [8]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)
train_year = train_year.drop('year', axis=1, inplace=False)

In [9]:
train_data= pd.merge(train_year,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [10]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [11]:
hist.head(2)

,계좌 ID,기준일자,종목코드,잔고수량,잔고금액,주당 액면가
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-08-20,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-06-23,A008770,20.0,1390000.0,5000.0


In [12]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [13]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [14]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [15]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [16]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 15.3397	valid_0's l2: 235.307
[1000]	valid_0's rmse: 15.2265	valid_0's l2: 231.846
Early stopping, best iteration is:
[778]	valid_0's rmse: 15.1257	valid_0's l2: 228.786
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 2.8472	valid_0's l2: 8.10657
[1000]	valid_0's rmse: 2.56427	valid_0's l2: 6.5755
Early stopping, best iteration is:
[925]	valid_0's rmse: 2.54008	valid_0's l2: 6.45199
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 2.50008	valid_0's l2: 6.25038
[1000]	valid_0's rmse: 2.01351	valid_0's l2: 4.05421
[1500]	valid_0's rmse: 1.91071	valid_0's l2: 3.65081
[2000]	valid_0's rmse: 1.88215	valid_0's l2: 3.54247
Did not meet early stopping. Best iteration is:
[1998]	valid_0's rmse: 1.88209	valid_0's l2: 3.54226
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 2.37887	valid_0's l2: 5.65903
[1000]

In [17]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [18]:
predict

array([828.28265679, 828.16741041, 827.90559872, ..., 823.36436112,
       821.08413307, 823.75633584])

In [19]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [20]:
submission["hold_d"] = np.round(predict)

In [21]:
submission.to_csv('sub2_경제_lgbm.csv', header=True, index=False, encoding='cp949')

## 연도별로 차이 조금만 ㅠㅠ

In [25]:
train_eco['year']= train_eco['매수일자'].apply(lambda x: x[0:4])

In [26]:
train_eco['과거 보유일'] = np.nan

train_eco2016 = train_eco[train_eco['year']=='2016'].copy()
train_eco2016['과거 보유일'] = train_eco2016['보유기간(일)']*0.5

train_eco2017 = train_eco[train_eco['year']=='2017'].copy()
train_eco2017['과거 보유일'] = train_eco2017['보유기간(일)']*0.55

train_eco2018 = train_eco[train_eco['year']=='2018'].copy()
train_eco2018['과거 보유일'] = train_eco2018['보유기간(일)']*0.6

train_eco2019 = train_eco[train_eco['year']=='2019'].copy()
train_eco2019['과거 보유일'] = train_eco2019['보유기간(일)']*0.65

train_eco2020 = train_eco[train_eco['year']=='2020'].copy()
train_eco2020['과거 보유일'] = train_eco2020['보유기간(일)']*0.7

In [27]:
train_year =pd.concat([train_eco2016,train_eco2017,train_eco2018,train_eco2019,train_eco2020], ignore_index=True)

In [28]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)
train_year = train_year.drop('year', axis=1, inplace=False)

In [29]:
train_data= pd.merge(train_year,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [30]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [31]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [32]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [33]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [34]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [35]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 11.3934	valid_0's l2: 129.809
Early stopping, best iteration is:
[687]	valid_0's rmse: 11.0602	valid_0's l2: 122.328
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 2.08371	valid_0's l2: 4.34184
[1000]	valid_0's rmse: 1.80467	valid_0's l2: 3.25682
Early stopping, best iteration is:
[1071]	valid_0's rmse: 1.80054	valid_0's l2: 3.24196
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.68977	valid_0's l2: 2.85533
[1000]	valid_0's rmse: 1.36152	valid_0's l2: 1.85374
[1500]	valid_0's rmse: 1.25843	valid_0's l2: 1.58364
[2000]	valid_0's rmse: 1.21418	valid_0's l2: 1.47424
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 1.2141	valid_0's l2: 1.47405
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.55833	valid_0's l2: 2.42841
[1000]	valid_0's rmse: 1.3663	valid_0's l2: 1.86677
Early

In [36]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [37]:
predict

array([812.46421391, 800.11151477, 802.65279991, ..., 801.27180093,
       799.07530899, 795.86660427])

In [38]:
# submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [20]:
# submission["hold_d"] = np.round(predict)

In [21]:
# submission.to_csv('sub2_경제_lgbm.csv', header=True, index=False, encoding='cp949')

## 연도별로 조금 줄이고...

In [45]:
train_eco = pd.read_csv("train_경제지표.csv", encoding='cp949')
train_eco['year']= train_eco['매수일자'].apply(lambda x: x[0:4])

In [46]:
train_eco['과거 보유일'] = np.nan

train_eco2016 = train_eco[train_eco['year']=='2016'].copy()
train_eco2016['과거 보유일'] = train_eco2016['보유기간(일)']*0.5

train_eco2017 = train_eco[train_eco['year']=='2017'].copy()
train_eco2017['과거 보유일'] = train_eco2017['보유기간(일)']*0.525

train_eco2018 = train_eco[train_eco['year']=='2018'].copy()
train_eco2018['과거 보유일'] = train_eco2018['보유기간(일)']*0.55

train_eco2019 = train_eco[train_eco['year']=='2019'].copy()
train_eco2019['과거 보유일'] = train_eco2019['보유기간(일)']*0.575

train_eco2020 = train_eco[train_eco['year']=='2020'].copy()
train_eco2020['과거 보유일'] = train_eco2020['보유기간(일)']*0.6

In [47]:
train_year =pd.concat([train_eco2016,train_eco2017,train_eco2018,train_eco2019,train_eco2020], ignore_index=True)

In [48]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)
train_year = train_year.drop('year', axis=1, inplace=False)

In [49]:
train_data= pd.merge(train_year,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [50]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [51]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [52]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [53]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [54]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [55]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 10.0373	valid_0's l2: 100.747
[1000]	valid_0's rmse: 9.64962	valid_0's l2: 93.1151
[1500]	valid_0's rmse: 9.67458	valid_0's l2: 93.5975
Early stopping, best iteration is:
[1261]	valid_0's rmse: 9.64669	valid_0's l2: 93.0585
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.63744	valid_0's l2: 2.68121
[1000]	valid_0's rmse: 1.42245	valid_0's l2: 2.02337
[1500]	valid_0's rmse: 1.41511	valid_0's l2: 2.00252
Early stopping, best iteration is:
[1473]	valid_0's rmse: 1.41473	valid_0's l2: 2.00147
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.11769	valid_0's l2: 1.24923
[1000]	valid_0's rmse: 0.839044	valid_0's l2: 0.703994
[1500]	valid_0's rmse: 0.797158	valid_0's l2: 0.635461
[2000]	valid_0's rmse: 0.777404	valid_0's l2: 0.604357
Did not meet early stopping. Best iteration is:
[1993]	valid_0's rmse: 0.777382	valid_0's l2: 0.604323
Tr

In [56]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [57]:
predict

array([786.53642972, 765.60289347, 764.55299342, ..., 758.44293294,
       751.02431709, 763.99008309])

## 가중치 반대로

In [59]:
train_eco = pd.read_csv("train_경제지표.csv", encoding='cp949')
train_eco['year']= train_eco['매수일자'].apply(lambda x: x[0:4])

In [60]:
train_eco['과거 보유일'] = np.nan

train_eco2016 = train_eco[train_eco['year']=='2016'].copy()
train_eco2016['과거 보유일'] = train_eco2016['보유기간(일)']*0.7

train_eco2017 = train_eco[train_eco['year']=='2017'].copy()
train_eco2017['과거 보유일'] = train_eco2017['보유기간(일)']*0.65

train_eco2018 = train_eco[train_eco['year']=='2018'].copy()
train_eco2018['과거 보유일'] = train_eco2018['보유기간(일)']*0.6

train_eco2019 = train_eco[train_eco['year']=='2019'].copy()
train_eco2019['과거 보유일'] = train_eco2019['보유기간(일)']*0.55

train_eco2020 = train_eco[train_eco['year']=='2020'].copy()
train_eco2020['과거 보유일'] = train_eco2020['보유기간(일)']*0.5

In [61]:
train_year =pd.concat([train_eco2016,train_eco2017,train_eco2018,train_eco2019,train_eco2020], ignore_index=True)

In [62]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)
train_year = train_year.drop('year', axis=1, inplace=False)

In [63]:
train_data= pd.merge(train_year,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [64]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [65]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [66]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [67]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [68]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [69]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 11.082	valid_0's l2: 122.812
[1000]	valid_0's rmse: 11.0682	valid_0's l2: 122.506
Early stopping, best iteration is:
[796]	valid_0's rmse: 11.037	valid_0's l2: 121.815
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 2.00656	valid_0's l2: 4.0263
[1000]	valid_0's rmse: 1.88836	valid_0's l2: 3.5659
Early stopping, best iteration is:
[929]	valid_0's rmse: 1.88115	valid_0's l2: 3.53873
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.83698	valid_0's l2: 3.3745
[1000]	valid_0's rmse: 1.60194	valid_0's l2: 2.56623
[1500]	valid_0's rmse: 1.51949	valid_0's l2: 2.30884
[2000]	valid_0's rmse: 1.48237	valid_0's l2: 2.19742
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 1.48237	valid_0's l2: 2.19742
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.94333	valid_0's l2: 3.77653
[1000]	va

In [70]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [71]:
predict

array([757.37111239, 716.38747406, 726.70232886, ..., 685.79821799,
       679.98769496, 663.99059385])

## dacon baseline으로 

In [40]:
train_eco = pd.read_csv("train_경제지표.csv", encoding='cp949')
test = pd.read_csv("test_경제지표.csv", encoding='cp949')

In [41]:
train_data= pd.merge(train_eco,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [42]:
train_data['과거 보유일'] = train_data['보유기간(일)']*0.6
train_data['과거 보유일'] = round(train_data['과거 보유일'],2)

In [43]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [44]:
hist["stk_p"] = hist["잔고금액"] / hist["잔고수량"]
hist = hist.fillna(0)

In [45]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [46]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [47]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [48]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [49]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 3.2675	valid_0's l2: 10.6765
[1000]	valid_0's rmse: 2.90124	valid_0's l2: 8.41718
[1500]	valid_0's rmse: 2.7368	valid_0's l2: 7.49006
[2000]	valid_0's rmse: 2.65075	valid_0's l2: 7.02649
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 2.65073	valid_0's l2: 7.02635
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.787695	valid_0's l2: 0.620463
[1000]	valid_0's rmse: 0.646286	valid_0's l2: 0.417685
Early stopping, best iteration is:
[1103]	valid_0's rmse: 0.63585	valid_0's l2: 0.404306
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.3673	valid_0's l2: 1.86951
Early stopping, best iteration is:
[503]	valid_0's rmse: 1.36638	valid_0's l2: 1.86699
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.317914	valid_0's l2: 0.101069
[1000]	valid_0's rmse: 0.231972	valid_0's l2: 0.0538

In [50]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [51]:
predict

array([13.00094445, 13.00094595, 13.00056156, ..., 26.98711057,
       26.98711057, 26.9873482 ])

In [85]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [86]:
submission["hold_d"] = np.round(predict)

In [87]:
submission.to_csv('sub4_경제_lgbm_가중치0.6으로 동일.csv', header=True, index=False, encoding='cp949')

# 외부경제 데이터 정규화

In [6]:
train_eco_scaled = pd.read_csv('train_eco_scaled.csv', encoding='cp949')

In [7]:
train_eco_scaled['year']= train_eco_scaled['매수일자'].apply(lambda x: x[0:4])

In [8]:
train_eco_scaled

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가,광물가격,기준금리,환율,year
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,0.931539,0.532362,0.8,0.357053,2018
1,f431b2a7651bccfc7ce8f294fdacdf0b7b31db734be701...,A023590,2018-07-26,5,0.931539,0.532362,0.8,0.357053,2018
2,43a0eb89f16d3a46767794dda9c31cd15e1ff9daf33332...,A010820,2018-07-26,3,0.931539,0.532362,0.8,0.357053,2018
3,5b6973bff9ab839507cf2f839b8b6d190d8ac2144fa5a4...,A005930,2018-07-26,122,0.931539,0.532362,0.8,0.357053,2018
4,1c52768eebea2eab6a7ed8cc95d838a3732fc6dce51ba3...,A006360,2018-07-26,2,0.931539,0.532362,0.8,0.357053,2018
...,...,...,...,...,...,...,...,...,...
681467,bcbf85d4382788cc37e75b11ab141e3bf7cde87ff7253d...,A267250,2019-03-27,89,0.839770,0.552473,1.0,0.355557,2019
681468,6d78a1da2ff49b7be5a15a22134c743e0ed478f58bb6f5...,A253450,2019-03-27,7,0.839770,0.552473,1.0,0.355557,2019
681469,ed62927d381883a19affcc2097336e801e2212ba0cc56d...,A091990,2019-03-27,4,0.839770,0.552473,1.0,0.355557,2019
681470,5eafe957081039a4c9e71186cdd1c74dfd674f00be01cc...,A034940,2019-03-27,1,0.839770,0.552473,1.0,0.355557,2019


In [9]:
train_eco_scaled['과거 보유일'] = np.nan

train_eco_scaled2016 = train_eco[train_eco_scaled['year']=='2016'].copy()
train_eco_scaled2016['과거 보유일'] = train_eco_scaled2016['보유기간(일)']*0.6

train_eco_scaled2017 = train_eco_scaled[train_eco_scaled['year']=='2017'].copy()
train_eco_scaled2017['과거 보유일'] = train_eco_scaled2017['보유기간(일)']*0.6

train_eco_scaled2018 = train_eco_scaled[train_eco_scaled['year']=='2018'].copy()
train_eco_scaled2018['과거 보유일'] = train_eco_scaled2018['보유기간(일)']*0.6

train_eco_scaled2019 = train_eco_scaled[train_eco_scaled['year']=='2019'].copy()
train_eco_scaled2019['과거 보유일'] = train_eco_scaled2019['보유기간(일)']*0.6

train_eco_scaled2020 = train_eco_scaled[train_eco_scaled['year']=='2020'].copy()
train_eco_scaled2020['과거 보유일'] = train_eco_scaled2020['보유기간(일)']*0.6

In [10]:
train_year =pd.concat([train_eco_scaled2016,train_eco_scaled2017,train_eco_scaled2018,train_eco_scaled2019,train_eco_scaled2020], ignore_index=True)

In [11]:
train_year['과거 보유일'] = round(train_year['과거 보유일'],2)
train_year = train_year.drop('year', axis=1, inplace=False)

In [12]:
train_data= pd.merge(train_year,iem, how='left', on='종목코드')
train_data = pd.merge(train_data, cus,how='left', on='계좌 ID')

test_data= pd.merge(test, iem, on='종목코드')
test_data = pd.merge(test_data , cus, on='계좌 ID')

In [13]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [14]:
hist

,계좌 ID,기준일자,종목코드,잔고수량,잔고금액,주당 액면가
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-08-20,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-06-23,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2016-01-04,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-08-14,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-06-23,A005930,20.0,1028000.0,100.0
...,...,...,...,...,...,...
2573834,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,2020-08-06,A035720,1.0,364000.0,500.0
2573835,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,2020-08-13,A035720,0.0,0.0,500.0
2573836,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,2020-08-19,A035720,1.0,376500.0,500.0
2573837,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,2020-08-25,A035720,0.0,0.0,500.0


In [15]:
hist["stk_p"] = hist["잔고금액"] / hist["잔고수량"]
hist = hist.fillna(0)

In [16]:
train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])] # 매수일 = 기준일자
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1) # 계좌번호, 종목코드, 매수일, 기준일자
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

In [17]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"]) # 종목 한글명
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [18]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [19]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [20]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 7.61818	valid_0's l2: 58.0367
[1000]	valid_0's rmse: 7.60853	valid_0's l2: 57.8898
Early stopping, best iteration is:
[738]	valid_0's rmse: 7.55514	valid_0's l2: 57.0801
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.792905	valid_0's l2: 0.628699
[1000]	valid_0's rmse: 0.584282	valid_0's l2: 0.341385
Early stopping, best iteration is:
[969]	valid_0's rmse: 0.584013	valid_0's l2: 0.341072
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.392125	valid_0's l2: 0.153762
[1000]	valid_0's rmse: 0.189713	valid_0's l2: 0.0359912
Early stopping, best iteration is:
[1036]	valid_0's rmse: 0.189651	valid_0's l2: 0.0359674
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.426481	valid_0's l2: 0.181886
[1000]	valid_0's rmse: 0.222211	valid_0's l2: 0.0493777
[1500]	valid_0's rmse: 0.221926	valid_0's l2: 0.049

In [21]:
result = []
for i in models:
    result.append(i.predict(test_data))
    
predict = np.mean(result, axis = 0)

In [22]:
predict

array([811.31607917, 757.30599724, 750.98071861, ..., 775.995573  ,
       764.53630377, 762.43212863])

In [23]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [24]:
submission["hold_d"] = np.round(predict)

In [26]:
submission.to_csv('sub6_경제scaled_lgbm_가중치0.6으로 동일_변수추가.csv', header=True, index=False, encoding='cp949')